## Deliverable 2. Create a Customer Travel Destinations Map.

In [150]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [151]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
city_data_df.head(10)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Sakakah,29.9697,40.2064,60.80,20,0,11.50,clear sky,SA
1,1,Rikitea,-23.1203,-134.9692,78.60,70,9,5.68,clear sky,PF
2,2,Rreshen,41.7675,19.8756,39.20,87,20,6.91,few clouds,AL
3,3,Lasa,46.6166,10.7002,36.00,74,75,6.91,light snow,IT
4,4,Tasiilaq,65.6145,-37.6368,32.00,93,90,24.16,light snow,GL
5,5,Fairbanks,64.8378,-147.7164,48.20,46,90,4.61,haze,US
6,6,Airai,-8.9266,125.4092,71.06,43,64,3.00,broken clouds,TL
7,7,Talcahuano,-36.7167,-73.1167,59.00,77,0,6.91,clear sky,CL
8,8,Saint-Philippe,-21.3585,55.7679,73.40,83,40,5.75,light rain,RE
9,9,Pisco,-13.7000,-76.2167,66.20,82,0,11.50,clear sky,PE


In [152]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
lowest_temp = input("Enter lowest temperature: ")
minimum_temp = float(lowest_temp)

highest_temp = input("Enter maximum temperature: ")
maximum_temp = float(highest_temp)


Enter lowest temperature: 50
Enter maximum temperature: 90


In [164]:
type(minimum_temp)


float

In [165]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
temp_range = city_data_df.loc[((city_data_df['Max Temp']) >= minimum_temp) & ((city_data_df['Max Temp']) < maximum_temp)]




In [166]:
# 4a. Determine if there are any empty rows.
temp_range.isnull().sum()



City_ID                0
City                   0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
Country                5
dtype: int64

In [167]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
city_data_temp_range = temp_range.dropna()
city_data_range_df = pd.DataFrame(city_data_temp_range)
city_data_range_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Country
0,0,Sakakah,29.9697,40.2064,60.80,20,0,11.50,clear sky,SA
1,1,Rikitea,-23.1203,-134.9692,78.60,70,9,5.68,clear sky,PF
6,6,Airai,-8.9266,125.4092,71.06,43,64,3.00,broken clouds,TL
7,7,Talcahuano,-36.7167,-73.1167,59.00,77,0,6.91,clear sky,CL
8,8,Saint-Philippe,-21.3585,55.7679,73.40,83,40,5.75,light rain,RE
...,...,...,...,...,...,...,...,...,...,...
648,648,Mayumba,-3.4320,10.6554,78.73,87,100,5.70,overcast clouds,GA
649,649,Manaus,-3.1019,-60.0250,80.60,89,0,4.61,clear sky,BR
652,652,Kualakapuas,-3.0091,114.3876,80.60,94,20,5.75,few clouds,ID
653,653,Guider,9.9342,13.9486,84.94,25,94,4.45,overcast clouds,CM


In [157]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = city_data_range_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Sakakah,SA,60.80,clear sky,29.9697,40.2064,
1,Rikitea,PF,78.60,clear sky,-23.1203,-134.9692,
6,Airai,TL,71.06,broken clouds,-8.9266,125.4092,
7,Talcahuano,CL,59.00,clear sky,-36.7167,-73.1167,
8,Saint-Philippe,RE,73.40,light rain,-21.3585,55.7679,
9,Pisco,PE,66.20,clear sky,-13.7000,-76.2167,
10,Lagoa,PT,60.69,scattered clouds,39.0500,-27.9833,
11,Rafai,CF,70.21,light rain,4.9500,23.9167,
12,Hermanus,ZA,63.00,broken clouds,-34.4187,19.2345,
14,Cidreira,BR,71.37,overcast clouds,-30.1811,-50.2056,


In [158]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
   

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
       
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    # Grab the first hotel from the results and store the name.   
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...skipping.")
        

Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.
Hotel not found...skipping.


In [159]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)

clean_hotel_df = hotel_df.dropna()


In [160]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [161]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Current Description</dt><dd>{Current Description} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
0,29.9697,40.2064
1,-23.1203,-134.9692
6,-8.9266,125.4092
7,-36.7167,-73.1167
8,-21.3585,55.7679
...,...,...
648,-3.4320,10.6554
649,-3.1019,-60.0250
652,-3.0091,114.3876
653,9.9342,13.9486


In [162]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))